# Assignement 2

In this assignement you will learn to make a robot move and avoid obstacles.

Objectives:
 - use a robot simulator: Gazebo
 - visualize robot sensors in Rviz
 - programm a simple obstacle avoidance algorithm
 - control a simulated robot
 - control a real robot
 - have fun

The expected result at the end of the assignement is 

![gazebo_rviz](static/img/screen_turtlebot_gazebo_rviz.png)
Expected result: https://youtu.be/nsdixTXz4V0



Ressources:
-  http://gazebosim.org/tutorials: official gazebo documentation


Tips:

### To open a new terminal: 

    xterm &
    
### To copy and paste in an xterminal:

    select what you want to copy with the mouse
    paste using the middle button (or use shift+insert shortcut)
    


## 1. Launch the robot simulator

- load a world
- spawn a robot in the world

Fortunately, it could be done in one command:

    TURTLEBOT_GAZEBO_WORLD_FILE=/opt/ros/kinetic/share/turtlebot_gazebo/worlds/corridor.world roslaunch turtlebot_gazebo turtlebot_world.launch




## 2. List all the availables topics

List all the availables topics and their types using:

    rostopic list or rostopic list -v
    


## 3. Visualize the /scan topic in a shell

Use the command rostopic echo to visualize the scan topic

Tip: Use the -n 1 option to display only one message

## 4. The content of the /scan messages

What is the type of the /scan topic ?  (tip: rostopic info topic_name)



Use the command rosmsg to get more information concerning this type of message:

    rosmsg type_name
    
Tip: Use the -r option to display the comments of each field

On the following message what is the meaning of the value in bold

<pre> 
header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: /camera_depth_frame
angle_min: <b>-0.521567881107</b>
angle_max: 0.524276316166
angle_increment: <b>0.00163668883033</b>
time_increment: 0.0
scan_time: 0.0329999998212
range_min: 0.449999988079
range_max: <b>10.0</b>
ranges: [<b>2.0153989791870117</b>, <b>2.0138657093048096</b>, 2.012336492538452, 2.0108108520507812, 2.0077707767486572, 2.006256341934204, 2.0047457218170166, 2.0032386779785156, 2.0017354488372803, 2.0002357959747314, 1.9972481727600098, 1.9957600831985474, 1.9942755699157715, 1.9927947521209717, 1.9913179874420166, 1.9883757829666138, 1.9869105815887451, 1.9854487180709839, 1.9839911460876465, 1.9825372695922852, 1.9810874462127686, 1.9781988859176636, 1.9767606258392334, 1.9753262996673584, 1.973895788192749, 1.9724690914154053, 1.9710463285446167, 1.9696277379989624, 1.9668021202087402, <b>nan</b>, 1.9639922380447388, 1.9625933170318604, 1.9611984491348267, 1.9598076343536377, 1.958420753479004, 1.9556587934494019, 1.9542839527130127, 1.9529131650924683, 1.9515464305877686, 1.950183629989624, 1.9488251209259033, 1.9474706649780273, 1.946120262145996, 1.9434314966201782, 1.942093

</pre>

How could you use this information to build a small application that makes the robot move forward until there is an obstacle in 5 meter distance. (use pseudo-code)

## 5. Visualize the robot sensors data in Rviz

Run the ros visualization tool using the following command

    rosrun rviz rviz

### a) Set the fixed frame

Set the fixed frame to /base_link



### b) Add a robot model


### c) Add a laser scan visualization that display the /scan topic


To get a color that represent the distance to the obstacle you can use the Color transformer intensity on the x axis.

### d) Add a pointcloud2 visualization that display the 3d camera pointcloud


### e) Move the robot in gazebo and check that the visualization works as expected


## 6. Send command to move the robot

As with the turlesimulator you can send command to the robot using a cmd_vel topic.

use the following command to make the robot move forward

     rostopic pub /cmd_vel_mux/input/teleop geometry_msgs/Twist "linear:
      x: 1.0
      y: 0.0
      z: 0.0
    angular:
      x: 0.0
      y: 0.0
      z: 0.0"

## 7. Write an obstacle detection function

Complete the following python code so that the function return True if there is a front obstacle in the scan message in less than 4.0 meters.

    Tip use the np.nanmin, np.nanmean, np.nanmax function

In [2]:
# !!!
from sensor_msgs.msg import LaserScan

def is_obstacle_front(laser_scan_msg, obstacle_threshold_distance=4.0):
    res = False
    my_min = np.nanmin(laser_scan_msg.ranges)
    if  my_min > obstacle_threshold_distance:
        return False
    return True


Test your function on the two following message

In [3]:
from sensor_msgs.msg import LaserScan
import numpy as np

laser_scan_msg_without_obstacle = LaserScan()
laser_scan_msg_without_obstacle.ranges = [np.nan, np.nan, 8, 8, 9, 7, 8, np.nan, 7]

laser_scan_msg_with_obstacle = LaserScan()
laser_scan_msg_with_obstacle.ranges = [3, np.nan, 1, 2, 1, 3, 8, np.nan, 3]


In [4]:
is_obstacle_front(laser_scan_msg_with_obstacle)

True

In [7]:
# !!!
is_obstacle_front(laser_scan_msg_without_obstacle)

False

### 7.bis) Write a function that return a forward move cmd if there is no obstacle


Using the previous function write a function that return a cmd (Twist message) that represent a forward move if there is no front obstacle, and a stop command if there is an obstacle. 

In [8]:
# !!!
from geometry_msgs.msg import Twist

def compute_move_cmd(laser_scan_msg, obstacle_threshold_distance=5.0, max_speed=0.1):
    res = Twist()
    if is_obstacle_front(laser_scan_msg, obstacle_threshold_distance=obstacle_threshold_distance):
        res.linear.x = 0.0
    else:
        res.linear.x = max_speed
    
    return res
    
    

Test your function with the two following LaserScan message:

In [9]:
from sensor_msgs.msg import LaserScan
import numpy as np



# TEST YOUR FUNCTION HERE !!! 

## !!!

print(compute_move_cmd(laser_scan_msg_with_obstacle))
print(compute_move_cmd(laser_scan_msg_without_obstacle))

linear: 
  x: 0.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: 0.1
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0


### 8) Test your function on the robot

- Use the notebook to test your function on the simulated robot [move_until_obstacle_node](move_until_obstacle_node.ipynb)


### 9) Now you are going to write an exploration function

Write a function that return: 

Move forward if there is no front obstacle
Turn Left if there is a front obstacle 




In [11]:
from geometry_msgs.msg import Twist

# !!!
def compute_move_cmd(laser_scan_msg, obstacle_threshold_distance=5.0, max_speed=0.1):
    res = Twist()
    if is_obstacle_front(laser_scan_msg, obstacle_threshold_distance=obstacle_threshold_distance):
        res.linear.x = 0.0
        res.angular.z = 0.1
    else:
        res.linear.x = max_speed
    
    return res
    

### 9 bis): test your function on the robot

- Use the notebook to test your function on the simulated robot [move_until_obstacle_node](move_until_obstacle_node.ipynb)


### Bonus 1: Improve the previous function so your robot could explore all the environment

Some ideas: Take a right turn if there is less obstacle on the left than the right etc..

### Bonus 2: Test on the real turtleBot